# **Initialization** 

In [1]:
import pandas as pd
import numpy as np
import torch

### Control

In [19]:
# Paths
RAW_DATA_FOLDER = 'raw_data'
TARGET_FILE_PATH = 'unprocessed_data'

# Flow Controls
RELOAD_RAW_DATA = False
DO_SMOTE = True
DATA = 'insomnia'  # Options: 'depression', 'insomnia', 'electrical_circuit'

# System variables
STATE = 42
TEST_SET_FRACTION = 0.20
MISSING_VALUES_THRESHOLD = 0.50
SAMPLES_ELECTRICAL_CIRCUIT = 5000
VERBOSE = True
FLIP_LABEL_FRACTION = 0.03

np.random.seed(STATE)

# **Data Preparation**

### Merge raw data files

In [20]:
from raw_data_loader import load_raw_data

if (RELOAD_RAW_DATA):
    load_raw_data(RAW_DATA_FOLDER, TARGET_FILE_PATH)

### Preprocessing and Split

In [21]:
from preprocessing_depression import clean_and_preprocess_depression_data
from preprocessing_insomnia import clean_and_preprocess_insomnia_data
from preprocessing_electrical_circuit import gen_and_preprocess_ec_data

dataset = pd.read_csv(TARGET_FILE_PATH + '/depression_data.csv')

if DATA == 'depression':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_depression_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'insomnia':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = clean_and_preprocess_insomnia_data(dataset, RAW_DATA_FOLDER, TEST_SET_FRACTION, STATE, MISSING_VALUES_THRESHOLD)
elif DATA == 'electrical_circuit':
    X_train, X_test, y_train, y_test, y_embed_train, y_embed_test = gen_and_preprocess_ec_data(SAMPLES_ELECTRICAL_CIRCUIT, TEST_SET_FRACTION, STATE)
    DO_SMOTE = False
else:
    raise ValueError("Invalid dataset selected")

#TODO Fix issue with the time columns SLQ300/310/320/330 in depression and processing

0    2794
1    1329
Name: count, dtype: int64


c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:704: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_moderate_LTPA"] = data["PAD790Q"] * data["PAD790U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:705: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_vigorous_LTPA"] = data["PAD810Q"] * data["PAD810U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia

True
True
True
True
True
True
True
True


c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:704: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_moderate_LTPA"] = data["PAD790Q"] * data["PAD790U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia.py:705: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["approximate_freq_vigorous_LTPA"] = data["PAD810Q"] * data["PAD810U"].map(convert_frequency)
c:\Users\jaspe\Documents\Github_projects\DSAIT4020-A2\preprocessing_insomnia

In [22]:
if DATA == "depression":
    invalid_row_mask = y_embed_train.isin([7, 9]).any(axis=1)
    print(y_embed_train[invalid_row_mask])
    assert len(y_embed_train[invalid_row_mask]) == 0, "There are targets with a 'refused' or 'unknown' value"

if DATA == "insomnia":
    invalid_row_mask = y_embed_train.isna().any(axis=1)
    print(y_embed_train[invalid_row_mask])
    assert len(y_embed_train[invalid_row_mask]) == 0, "There are targets with a 'refused' or 'unknown' value"


      SLQ300  SLQ310  SLD012  SLQ320  SLQ330  SLD013
1247    60.0   570.0     8.0   600.0   600.0     NaN
3399    60.0   330.0     4.0     NaN     NaN     2.0
1902    60.0   300.0     4.0     NaN     NaN    14.0
3500   360.0   750.0     6.0     NaN     NaN    14.0
1748     NaN     NaN     NaN     NaN     NaN     NaN
...      ...     ...     ...     ...     ...     ...
4089  1260.0   240.0     7.0     NaN     NaN     NaN
2880  1380.0   360.0     7.0     NaN     NaN     NaN
3149     NaN     NaN     NaN     NaN     NaN     NaN
469      NaN     NaN     2.0   240.0   540.0     5.0
1181     NaN     NaN    14.0     0.0   840.0    14.0

[84 rows x 6 columns]


AssertionError: There are targets with a 'refused' or 'unknown' value

### Data Balancing

In [6]:
from data_balancing import resample_training_data

if DO_SMOTE:
    X_train, y_train, y_embed_train = resample_training_data(X_train, y_train, y_embed_train, random_state=STATE)

In [7]:
# Check class distribution
classes, counts = np.unique(y_train, return_counts=True)
print("Class Distribution:\n", dict(zip(classes, counts)))

if len(classes) > 1:
    class_ratio = counts[1] / counts[0]
    print(f"\nClass ratio: {class_ratio:.3f}")
else:
    print("\nOnly one class present.")

Class Distribution:
 {np.int64(0): np.int64(3104), np.int64(1): np.int64(3024)}

Class ratio: 0.974


### Introduce Noise to label

In [8]:
assert FLIP_LABEL_FRACTION > 0.0 and FLIP_LABEL_FRACTION < 1.0, "FLIP_LABEL_FRACTION should be beween 0.0 and 1.0"

# Randomly select indices to flip
if FLIP_LABEL_FRACTION > 0.0:
    num_to_flip = int(FLIP_LABEL_FRACTION * len(y_train))
    flip_indices = np.random.choice(len(y_train), size=num_to_flip, replace=False)

    # If y_train is a pandas Series, convert to int for safe arithmetic
    if hasattr(y_train, 'iloc'):
        y_train = y_train.astype(int)
        y_train.iloc[flip_indices] = 1 - y_train.iloc[flip_indices]
    else:  # numpy array
        y_train[flip_indices] = 1 - y_train[flip_indices]

### Make everything a numpy array

In [9]:
X_train = X_train.values if hasattr(X_train, "values") else np.array(X_train)
X_test = X_test.values if hasattr(X_test, "values") else np.array(X_test)

y_train = y_train.values.ravel() if hasattr(y_train, "values") else np.array(y_train).ravel()
y_test = y_test.values.ravel() if hasattr(y_test, "values") else np.array(y_test).ravel()

y_embed_train = y_embed_train.values if hasattr(y_embed_train, "values") else np.array(y_embed_train)
y_embed_test = y_embed_test.values if hasattr(y_embed_test, "values") else np.array(y_embed_test)

assert(isinstance(X_train, np.ndarray))
assert(isinstance(X_test, np.ndarray))
assert(isinstance(y_train, np.ndarray))
assert(isinstance(y_test, np.ndarray))
assert(isinstance(y_embed_train, np.ndarray))
assert(isinstance(y_embed_test, np.ndarray))

# **Models**

In [10]:
from baseline_models import train_multitarget_baseline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

### Training Bayesian Models

In [11]:
nb_model = GaussianNB()
y_pred_nb, acc_nb = train_multitarget_baseline(
                            model=nb_model,
                            is_classifier=False,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)



 ######################################## GaussianNB Multitarget Regressor ########################################
Train MSE per embedding: [1.68390992 1.58730418 1.54471279 1.41465405 1.74706266 1.67036554
 2.33518277 4.37679504 4.94941253]
Test MSE per embedding: [2.63151515 2.34060606 1.98060606 1.63030303 2.49939394 2.39393939
 2.98787879 4.88727273 5.86424242]
Average train MSE: 2.3677110530896432
Average test MSE: 3.023973063973064


### Training Random Forests

In [12]:
# Parameter
N_ESTIMATORS = 100

In [13]:
rf_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, random_state=STATE, n_jobs=-1)
y_pred_rf, mse_rf = train_multitarget_baseline(
                                    model=rf_model, 
                                    is_classifier=False, 
                                    X_train=X_train, 
                                    X_test=X_test, 
                                    y_embed_train=y_embed_train, 
                                    y_embed_test=y_embed_test,
                                    verbose=VERBOSE)



 ######################################## RandomForestRegressor Multitarget Regressor ########################################
Train MSE per embedding: [0.04868177 0.04020042 0.06631898 0.05258557 0.05571394 0.04533048
 0.04374636 0.0300101  0.01338597]
Test MSE per embedding: [0.71028897 0.58013855 0.91118412 0.73295661 0.83929721 0.61563006
 0.582684   0.37515891 0.17831927]
Average train MSE: 0.043997066289527244
Average test MSE: 0.6139619663299664


### Training Logistic Models

In [14]:
# Parameters
MAX_ITERATIONS = 1000

In [15]:
log_model = LogisticRegression(max_iter=MAX_ITERATIONS, class_weight='balanced', random_state=STATE)
y_pred_log, acc_log = train_multitarget_baseline(
                            model=log_model,
                            is_classifier=True,
                            X_train=X_train,
                            X_test=X_test,
                            y_embed_train=y_embed_train,
                            y_embed_test=y_embed_test,
                            verbose=VERBOSE)



 ######################################## LogisticRegression Multitarget Classifier ########################################
Train accuracy per embedding: [0.61684073 0.65943211 0.52529373 0.57865535 0.59252611 0.61471932
 0.6316906  0.64295039 0.71230418]
Test accuracy per embedding: [0.48363636 0.52727273 0.38545455 0.40727273 0.45212121 0.5430303
 0.53212121 0.63757576 0.7369697 ]
Average train accuracy: 0.6193791702930084
Average test accuracy: 0.5228282828282829


## Proposed MLPs

In [16]:
from proposed_models import train_joint_model, train_split_model, train_deep_joint_model, train_deep_split_model

DEVICE = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")
E_KEEP_RATE = 0.7
l = 1
if DATA == 'depression':
    l = 1e-2
elif DATA == 'insomnia':
    l = 1e-2
elif DATA == 'electrical_circuit':
    l = 1

EPOCHS = 100
AUGMENT_EPOCHS = EPOCHS//2
EARLY_STOP_EPOCHS = EPOCHS//5

In [17]:
# Sanity Checks
print("Using ", DEVICE, " for torch")

assert X_train.shape[0] >= 100 and y_train.shape[0] >= 100 and y_embed_train.shape[0] >= 100, "Arrays must have at least 100 samples for the check."

aligned = (len(X_train[:100]) == len(y_train[:100])) and (len(X_train[:100]) == len(y_embed_train[:100]))
assert aligned, "First 100 samples of X_train, y_train, and y_embed_train are not aligned."

Using  cpu  for torch


### Train Joint MLP

In [18]:
train_joint_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test,
                    e_kept_ratio=E_KEEP_RATE,
                    l=l,
                    epochs=EPOCHS,
                    augment_epochs=AUGMENT_EPOCHS,
                    early_stop_epochs=EARLY_STOP_EPOCHS,
                    device=DEVICE
                  )

KeyboardInterrupt: 

### Train Split MLP

In [ ]:
train_split_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test,
                    e_kept_ratio=E_KEEP_RATE,
                    epochs=EPOCHS,
                    augment_epochs=AUGMENT_EPOCHS,
                    early_stop_epochs=EARLY_STOP_EPOCHS,
                    device=DEVICE
                  )

Training:	###################################---------------	[70.0% - DONE]

================================   Split MLP   ================================
Regression Results:
MSE:	0.9559118747711182


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]
Training:	###-----------------------------------------------	[6.0%]

/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Training:	##################################################	[100.0%]

==========================   Split MLP (Augmented)   ==========================
Regression Results:
MSE:	0.9751307964324951


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]


/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### Train Deep Joint Model

In [ ]:
train_deep_joint_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test,
                        e_kept_ratio=E_KEEP_RATE,
                        l=l,
                        epochs=EPOCHS,
                        augment_epochs=AUGMENT_EPOCHS,
                        early_stop_epochs=EARLY_STOP_EPOCHS,
                        device=DEVICE
                      )

Training:	############--------------------------------------	[25.0% - DONE]

==============================   Deep Joint MLP   ==============================
Regression Results:
MSE:	1.1690424680709839


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]
Training:	###-----------------------------------------------	[6.0%]

/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Training:	####################------------------------------	[40.0% - DONE]

========================   Deep Joint MLP (Augmented)   ========================
Regression Results:
MSE:	1.1690424680709839


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]


/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


### Train Deep Split Model

In [ ]:
train_deep_split_model( X_train, X_test, y_train, y_test, y_embed_train, y_embed_test,
                        e_kept_ratio=E_KEEP_RATE,
                        epochs=EPOCHS,
                        augment_epochs=AUGMENT_EPOCHS,
                        early_stop_epochs=EARLY_STOP_EPOCHS,
                        device=DEVICE
                      )

Training:	######################################------------	[77.0% - DONE]

==============================   Deep Split MLP   ==============================
Regression Results:
MSE:	1.2037678956985474


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]
Training:	#####---------------------------------------------	[10.0%]

/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Training:	##################################################	[100.0%]

========================   Deep Split MLP (Augmented)   ========================
Regression Results:
MSE:	1.2037678956985474


Classification Results:
F1 score: 0.0
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       775
           1       0.00      0.00      0.00        59

    accuracy                           0.93       834
   macro avg       0.46      0.50      0.48       834
weighted avg       0.86      0.93      0.90       834

Confusion matrix:
 [[775   0]
 [ 59   0]]


/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
